# Robinet de Ransom 2D homogène

In [ ]:
from trustutils import run

run.introduction('A. Drouillet (TMA), J. Francescatto (CEA)')
run.description("Cas tests du robinet de Ramson avec découpage des phases GAZ et LIQUIDE.")


## Description du cas test
Le probléme de Ransom 1D a été présenté par V.H. Ransom dans. Ce cas test "raide" permet de tester la stabilité du schéma, sa précision et l'interaction avec des termes sources simples. Il représente l'écoulement 1D d'une colonne d'eau verticale soumise à la force de gravité. L'eau accélérant sous l'effet de la gravité, le diamétre du jet doit diminue pour assurer la conservation du débit. Le cas test consiste en une collone de 12 m de hauteur et 1 m de largeur contenant de l'eau et de l'air, traités comme des fluides incompressibles. Ces fluides obeissent à la loi d'état Stiffened Gas avec pour :
* l'air : Pinf=0 et gamma = 1.4
* l'eau : Pinf=6 10^8 et gamma = 4.4


Dans cette fiche de validation, il est regardé les résultats obtenus lorsque chaque phase (GAZ et LIQUIDE) est séparée en plusieurs sous-phases (4 pour GAZ et 8 pour LIQUIDE). Cela permet de valider la programmation informatique de la définition de sous phases dans le jeu de données. Cette base informatique servira pour la mise en place de d'une méthode multi groupes de type MUSIG.

In [ ]:
run.TRUST_parameters()

In [ ]:
from trustutils import run, plot
import os

run.reset()
run.initBuildDirectory()

liste_dis = ["VDF", "PolyMAC_P0"]
for dis in liste_dis:
    run.addCaseFromTemplate(f"jdd.data", dis, {"dis" : dis})
    run.addCaseFromTemplate(f"jdd_MUSIG.data", dis, {"dis" : dis})
    
run.printCases()
run.runCases()
run.tablePerf()

| | nb_Phases | nb_Proc |
| :---:| :---: | :---: |
| jdd ransom_StiffenedGas| 2 | 1 |
| jdd ransom_StiffenedGas MUSIG | 12 | 1 |

In [ ]:
# Approximate exact solution 
import math
import numpy as np

def exactAlphaGas(x,t):
    g = 10
    L = 12
    U0 = 10
    alpha0 = 0.2
    if(x<U0*t+0.5*g*t*t):
        return 1-((1-alpha0)*U0)/(math.sqrt(U0*U0+2*g*x))
    else:
        return alpha0

def exactVitesseLiquid(x,t):
    g = 10
    L = 12
    U0 = 10
    alpha0 = 0.2
    if(x<U0*t+0.5*g*t*t):
        return -math.sqrt(U0*U0+2*g*x)
    else:
        return -(U0+g*t)
    
def writeSolution(fichierName, Dt, function, N):
    L = 12
    xRange = np.linspace(0,L,N)
    fichier = open(fichierName, "w")
    for i in xRange :
        x=L-i
        ligne = str(x) + " " + str(function(i, Dt)) + " \n" 
        fichier.write(ligne)
    fichier.close()

In [ ]:
import time
from trustutils import plot
import matplotlib.pyplot as plt

#time step
timeStep = [0.3, 0.6, 0.8]
Ncases = len(timeStep)
for Dt in timeStep : 
    fichierNameAlpha = run.BUILD_DIRECTORY + "/AlphaExact" + str(Dt) + ".txt"
    fichierNameVitesse = run.BUILD_DIRECTORY+"/VitesseExact" + str(Dt) + ".txt"
    writeSolution(fichierNameAlpha, Dt, exactAlphaGas, 50)
    writeSolution(fichierNameVitesse, Dt, exactVitesseLiquid, 50)
    
a = plot.Graph("Resultats", nX=Ncases, nY=2)

line = 0

alpha_son = "SONDE_ALPHA_GAZ_HELIUM"
vit_son = "SONDE_VITESSE_LIQUIDE_SODIUM"

for t in timeStep:
    a.addPlot([line,0],"Taux de vide à t = %s" %t)
    
    for dis in liste_dis:
        a.addSegment(f"{dis}/jdd_{alpha_son}.son",lw=3,time=t,label=f"ransom {dis} nbPhases 2")
        for i in range (4):
            a.addSegment(f"{dis}/jdd_MUSIG_{alpha_son}_{i}.son",lw=3,time=t,label=f"ransom {dis} MUSIG nbPhases 12")
    
    data = plot.loadText("AlphaExact" + str(t) + ".txt")
    a.add(data[0],data[1],label="Analytique",lw=3,marker='o',color='k')
    a.label("y en m","Taux de vide")
    
    a.addPlot([line,1],"Vitesse du fluide à t = %s" %t)
    
    for dis in liste_dis:
        a.addSegment(f"{dis}/jdd_{vit_son}.son",lw=3,compo=1,time=t,label=f"ransom {dis} nbPhases 2")
        for i in range (8):
            a.addSegment(f"{dis}/jdd_MUSIG_{vit_son}_{i}.son",lw=3,compo=1,time=t,label=f"ransom {dis} MUSIG nbPhases 12")
        
    data = plot.loadText("VitesseExact" + str(t) + ".txt")
    a.add(data[0],data[1],label="Analytique",lw=3,marker='o',color='k')
    a.label("y en m","Vitesse du fluide m/s")
    line += 1   
  
# a.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=2)